In [80]:
import numpy as np
import pandas as pd
import numpy.ma as ma
from scipy.optimize import curve_fit
import numpy.linalg as LA
import math
import struct

#modulef = '/home/ubuntu/source/diff-classifier/diff_classifier/'
modulef = 'C:/Users/koolk/Desktop/diff-classifier/diff_classifier/'
import sys
sys.path.insert(0, modulef)

from utils import csv_to_pd
from msd import nth_diff, msd_calc, all_msds

In [140]:
d = {'Frame': [1, 2, 3, 4, 5],
             'X': [5, 6, 7, 8, 9],
             'Y': [6, 7, 8, 9, 10]}
df = pd.DataFrame(data=d)
df['MSDs'] = msd_calc(df)
df

,Frame,X,Y,MSDs
0,1,5,6,0.0
1,2,6,7,2.0
2,3,7,8,8.0
3,4,8,9,18.0
4,5,9,10,32.0


In [3]:
def alpha_calc(track):
    """
    Calculates the parameter alpha by fitting track MSD data to a function.
    
    Parameters
    ----------
    
    Returns
    ----------
    
    Examples
    ----------
    """

    x = track['Frame']
    y = track['MSDs']

    def msd_alpha(x, a, D):
        return 4*D*(x**a)

    try:
        popt, pcov = curve_fit(msd_alpha, x, y)
        a = popt[0]
        D = popt[1]
    except RuntimeError:
        print('Optimal parameters not found. Print NaN instead.')
        a = np.nan
        D = np.nan
    return a, D

In [42]:
def gyration_tensor(track):
    
    df = track
    
    Ta = np.sum((df['X'] - np.mean(df['X']))**2)/df['X'].shape[0]
    Tb = np.sum((df['Y'] - np.mean(df['Y']))**2)/df['Y'].shape[0]
    Tab = np.sum((df['X'] - np.mean(df['X']))*(df['X'] - np.mean(df['X'])))/df['X'].shape[0]

    w, v = LA.eig(np.array([[Ta, Tab], [Tab, Tb]]))
    l1 = max(w)
    l2 = min(w)
    return l1, l2

In [43]:
def asymmetry(track):

    l1, l2 = gyration_tensor(track)
    a1 = (l1**2 - l2**2)**2/(l1**2 + l2**2)**2
    a2 = l2/l1
    a3 = -np.log(1-((l1-l2)**2)/(2*(l1+l2)**2))

    return l1, l2, a1, a2, a3

In [45]:
l1, l2, a1, a2, a3 = asymmetry(df)

In [138]:
def aspectratio(track):
    rot_angle, area, width, height, center_point, corner_points = minBoundingRect(df)
    ar = width/height
    if ar > 1:
        counter = 1
    else:
        ar = 1/ar
    return ar

In [141]:
aspectratio(df)

6369051672525774.0

In [133]:
def minBoundingRect(df):
    
    #Based off of code from the following repo:
    # https://github.com/dbworth/minimum-area-bounding-rectangle/blob/master/python/min_bounding_rect.py

    df2 = np.zeros((df.shape[0]+1, 2))
    df2[:-1, :] = df[['X', 'Y']].values
    df2[-1, :] = df[['X', 'Y']].values[0, :]
    hull_points_2d = df2

    edges = np.zeros((len(hull_points_2d)-1, 2))

    for i in range(len(edges)):
        edge_x = hull_points_2d[i+1,0] - hull_points_2d[i,0]
        edge_y = hull_points_2d[i+1,1] - hull_points_2d[i,1]
        edges[i] = [edge_x,edge_y]

    edge_angles = np.zeros((len(edges)))

    for i in range(len(edge_angles)):
        edge_angles[i] = math.atan2( edges[i,1], edges[i,0] )
    edge_angles = np.unique(edge_angles)

    start_area = platform_c_maxint = 2 ** (struct.Struct('i').size * 8 - 1) - 1
    min_bbox = (0, start_area, 0, 0, 0, 0, 0, 0)
    for i in range( len(edge_angles) ):
        R = np.array([ [ math.cos(edge_angles[i]), math.cos(edge_angles[i]-(math.pi/2)) ],
                     [ math.cos(edge_angles[i]+(math.pi/2)), math.cos(edge_angles[i]) ] ])

        rot_points = np.dot(R, np.transpose(hull_points_2d) )

        min_x = np.nanmin(rot_points[0], axis=0)
        max_x = np.nanmax(rot_points[0], axis=0)
        min_y = np.nanmin(rot_points[1], axis=0)
        max_y = np.nanmax(rot_points[1], axis=0)

        width = max_x - min_x
        height = max_y - min_y
        area = width*height

        if (area < min_bbox[1]):
            min_bbox = (edge_angles[i], area, width, height, min_x, max_x, min_y, max_y)

    angle = min_bbox[0]   
    R = np.array([ [ math.cos(angle), math.cos(angle-(math.pi/2)) ], [ math.cos(angle+(math.pi/2)), math.cos(angle) ] ])
    proj_points = np.dot(R, np.transpose(hull_points_2d) ) # 2x2 * 2xn

    min_x = min_bbox[4]
    max_x = min_bbox[5]
    min_y = min_bbox[6]
    max_y = min_bbox[7]

    center_x = (min_x + max_x)/2
    center_y = (min_y + max_y)/2
    center_point = np.dot( [ center_x, center_y ], R )

    corner_points = np.zeros( (4,2) )
    corner_points[0] = np.dot( [ max_x, min_y ], R )
    corner_points[1] = np.dot( [ min_x, min_y ], R )
    corner_points[2] = np.dot( [ min_x, max_y ], R )
    corner_points[3] = np.dot( [ max_x, max_y ], R )

    return (angle, min_bbox[1], min_bbox[2], min_bbox[3], center_point, corner_points)
    # rot_angle, area, width, height, center_point, corner_points